# ОИАД. Лабораторная работа №3

## 1. Подготовка данных
1. привести категориальные признаки к числовым
2. вычислить парные корреляции признаков


In [18]:
import pandas as pd
import numpy as np

# Загружаем данные
train = pd.read_csv("../../datasets/insurance_train.csv")
test = pd.read_csv("../../datasets/insurance_test.csv")

# Преобразуем категориальные признаки в числовые
def encode_data(df):
    df = df.copy()
    df["sex"] = df["sex"].map({"male": 1, "female": 0})
    df["smoker"] = df["smoker"].map({"yes": 1, "no": 0})
    df = pd.get_dummies(df, columns=["region"], drop_first=True)
    return df

train_encoded = encode_data(train)
test_encoded = encode_data(test)

print("Пример обучающих данных после кодирования:\n", train_encoded.head())



Пример обучающих данных после кодирования:
    age  sex     bmi  children  smoker      charges  region_northwest  \
0   26    1  27.060         0       1  17043.34140             False   
1   58    1  36.955         2       1  47496.49445              True   
2   20    0  24.420         0       1  26125.67477             False   
3   51    0  38.060         0       1  44400.40640             False   
4   62    0  25.000         0       0  13451.12200             False   

   region_southeast  region_southwest  
0              True             False  
1             False             False  
2              True             False  
3              True             False  
4             False              True  


In [10]:
##Вычисляем парные корреляции признаков
corr = train_encoded.corr(numeric_only=True)
print("Корреляция признаков с целевой переменной 'charges':\n")
print(corr["charges"].sort_values(ascending=False))


Корреляция признаков с целевой переменной 'charges':

charges             1.000000
smoker              0.783519
age                 0.298395
bmi                 0.219566
children            0.069444
sex                 0.060221
region_southeast    0.009792
region_northwest   -0.032287
region_southwest   -0.053905
Name: charges, dtype: float64


Наибольшее влияние на цену медицинской страховки оказывают признаки курение, возраст и индекс массы тела (BMI). Пол, количество детей и регион оказывают слабое влияние.

## 2. Многомерная линейная регрессия
Построить модель линейной регрессии и подобрать параметры:
1. аналитически (реализовать самому)
2. численно, с помощью методов градиентного спуска (реализовать самому)


In [20]:
##Подготовка матриц признаков и целевой переменной
def prepare_X_y(df):
    X = df.drop(columns=["charges"]).copy()
    X = X.apply(pd.to_numeric, errors='coerce')  # гарантируем числовой тип
    X = np.c_[np.ones(len(X)), X.values.astype(np.float64)]  # добавляем столбец единиц
    y = df["charges"].values.reshape(-1, 1).astype(np.float64)
    return X, y

X_train, y_train = prepare_X_y(train_encoded)
X_test, y_test = prepare_X_y(test_encoded)



In [23]:
## Аналитическое решение (нормальное уравнение)
beta_analytical = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
print("Коэффициенты аналитически рассчитанной модели:\n", beta_analytical.flatten())



Коэффициенты аналитически рассчитанной модели:
 [-11458.00761083    256.71070712   -655.40504609    350.86390876
    483.10419606  23401.99257013   -200.5869865   -1389.89573219
  -1440.50759731]


In [24]:
##Численное решение (градиентный спуск)
def gradient_descent(X, y, lr=1e-5, epochs=50000):
    m, n = X.shape
    beta = np.zeros((n, 1))
    for i in range(epochs):
        grad = (1/m) * X.T @ (X @ beta - y)
        beta -= lr * grad
        if i % 10000 == 0:
            mse = np.mean((X @ beta - y) ** 2)
            print(f"Эпоха {i}: MSE = {mse:.2f}")
    return beta

beta_gd = gradient_descent(X_train, y_train, lr=1e-5, epochs=50000)
print("Коэффициенты модели (градиентный спуск):\n", beta_gd.flatten())



Эпоха 0: MSE = 350682814.48
Эпоха 10000: MSE = 141206520.96
Эпоха 20000: MSE = 137902774.23
Эпоха 30000: MSE = 134722494.40
Эпоха 40000: MSE = 131659675.48
Коэффициенты модели (градиентный спуск):
 [-133.97704509  212.99369513  124.15458081  182.68841042  169.54945435
 1968.65200326  -29.56708116  -72.39091075 -202.74111219]




* **Аналитическая модель**: курение, возраст и BMI сильнее всего влияют на страховку; остальные признаки мало влияют.
* **Градиентный спуск**: коэффициенты похожи по направлению, MSE постепенно уменьшается; требует нормализации и настройки скорости обучения.
* **Вывод**: основные факторы стоимости — **smoker, age, bmi**; аналитическая модель точна и стабильна, градиентный спуск применим для больших данных.



## 3. Оценка обобщающей способности
Сравнить между собой модели на тестовых данных по среднему квадрату ошибки:
1. константную - прогноз средним значением
2. из пункта 2

In [25]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# 1. Константная модель (среднее по train)
y_pred_const = np.full_like(y_test, y_train.mean())
mse_const = mse(y_test, y_pred_const)

# 2. Модель аналитическая
y_pred_analytical = X_test @ beta_analytical
mse_analytical = mse(y_test, y_pred_analytical)

# 3. Модель по градиентному спуску
y_pred_gd = X_test @ beta_gd
mse_gd = mse(y_test, y_pred_gd)

print(f"MSE константной модели: {mse_const:.2f}")
print(f"MSE аналитической модели: {mse_analytical:.2f}")
print(f"MSE модели градиентного спуска: {mse_gd:.2f}")


MSE константной модели: 141830094.36
MSE аналитической модели: 34216008.76
MSE модели градиентного спуска: 112681499.36


* **Константная модель**: 141 830 094 — наихудший результат, не учитывает признаки.
* **Аналитическая модель**: 34 216 008 — наилучший результат, точно предсказывает стоимость страховки.
* **Градиентный спуск**: 112 681 499 — лучше константной, но хуже аналитической; требует нормализации и настройки параметров для улучшения.

**Вывод:** аналитическая модель показывает наилучшую обобщающую способность, а ключевыми факторами стоимости являются **smoker, age, bmi**.
